# Imports

In [2]:
import pandas as pd
import numpy as np
import pickle
import requests
from bs4 import BeautifulSoup
import time
import re
import string
import pymongo

# Functions

## Scrape Links

In [3]:
def splitup(l, splitters):
    current = []
    for item in l:
        if item in splitters:
            yield current
            current = []
        else:
            current.append(item)
    yield current
    # generator object splitup
def sublist(l, splitters):
    return [subl for subl in splitup(l, splitters) if subl]

In [4]:
def all_links(judge):    
    t0 = time.time()
    response_delay = time.time() - t0
    time.sleep(10*response_delay)  # wait 10x longer than it took them to respond
    
    website = 'https://www.law.cornell.edu/supct/justices/%s.dec.html'
    url = website % judge
    header = {'User-Agent': 
              'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    soup =BeautifulSoup(requests.get(url, headers = header).text, "lxml")
    
    all_links =[]
    for link in soup.find_all('ul')[9].find_all('a'):
        try:
            all_links.append(link['name']) 
        except:
            try:
                writing = {}
                writing['name'] = link.text.strip()
                writing['cite'] = link.next_sibling.strip().replace(',', '').strip()
                writing['link'] = link['href']
                all_links.append(writing)
            except:
                pass
    
    splits =[]
    for link in soup.find_all('ul')[9].find_all('a', href=False):
        splits.append(str(link['name'])) 
    
    sublists_links = sublist(all_links, splits)
    del soup
    
    return sublists_links  

## Scrape Writings

In [5]:
def all_writings(judge):
    sublists = all_links(judge)
    subparts = {0: 'opinion', 1: 'concurrence', 2: 'dissent', 3: 'cdinpart'}
    
    for n in range(0, 4): #change it back to 4
        for writing in sublists[n]:       
            t0 = time.time()
            response_delay = time.time() - t0
            time.sleep(10*response_delay)  # wait 10x longer than it took them to respond
    
            website = 'https://www.law.cornell.edu/%s'
            url = website % writing['link']
            header = {'User-Agent': 
                  'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
            soup =BeautifulSoup(requests.get(url, headers = header).text, "lxml")
            
            writing['type'] = subparts[n] # added this after scraping Ginsburg, TEST
        
            try: # organized
                writing['text'] = str(soup.find_all('div', {'class' : subparts[n]})[0].text.strip()).replace('\n', ' ').replace('\t', '').replace('TOP', '')
            except: # disorganized
#                 pass
                try:
                    writing['text'] = dis_organized(soup)
                except:
                    pass
                
            print(writing['link'])
            del soup
            
    sub_list = [j for i in sublists for j in i] # added this after scraping Ginsburg, TEST
    return sub_list # added this after scraping Ginsburg, TEST

In [6]:
def dis_organized(soup):
    dirty = []
    for paragraphs in soup.find_all('div', {'id': 'content'})[0].find_all('p'):
        dirty.append(str(paragraphs.text.strip().replace('\n', ' ').replace('\t', '')))
    keepers = judges_part(dirty, 'TOP')  
    for paragraphs in keepers:
        paragraphs = str(paragraphs).replace("\'","").replace('\\', '')
    return keepers

In [7]:
def judges_part(l, splitters):
    all_opin = sublist(l, splitters)
    sub_parts = []
    for n in range(len(all_opin)):
        if str(judge).upper() in all_opin[n][0]: #<---------- judge VARIABLE SET?
            #NEXT STEP: if other judges names appear before judge name, skip.
            sub_parts.append(all_opin[n])
        else:
            pass
    parts = [j for i in sub_parts for j in i]
    return parts

## Scrape Counter

In [8]:
def text_counter(list_o_dict): #Ginsbot[0]
    count = 0
    for writing in list_o_dict:
        try:
            if writing['text']:
                count += 1
        except:
            pass
    print(count)

## Into MongoDB

In [9]:
def into_mongo(scrapings, judge):
    client = pymongo.MongoClient()
    db = client.supremes 
    db.create_collection(judge).insert(scrapings) # judge = "ginsburg"

# Super Scrape into Mongo Function

In [10]:
def scrape_into_mongo(judge):
    judge = judge
    scrapings = all_writings(judge)
    into_mongo(scrapings, judge)
    print(judge, "is done.")

In [1]:
scrape_into_mongo('scalia')

In [ ]:
scrape_into_mongo('ginsburg')